In [32]:
%load_ext autoreload
%autoreload 2
import gust  # library for loading graph data

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
import time
import random
from scipy.spatial.distance import squareform
torch.set_default_tensor_type('torch.cuda.FloatTensor')
%matplotlib inline
sns.set_style('whitegrid')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# Load the dataset using `gust` library
# graph.standardize() makes the graph unweighted, undirected and selects
# the largest connected component
# graph.unpack() returns the necessary vectors / matrices

A, X, _, y = gust.load_dataset('cora').standardize().unpack()
# A - adjacency matrix 
# X - attribute matrix - not needed
# y - node labels
A=A[:10,:10]

if (A != A.T).sum() > 0:
    raise RuntimeError("The graph must be undirected!")

if (A.data != 1).sum() > 0:
    raise RuntimeError("The graph must be unweighted!")
    
    
adj = torch.FloatTensor(A.toarray()).cuda()


In [34]:
# Make it stochastic
adj = torch.FloatTensor(A.toarray()).cuda()
'''
from the paper Sampling from Large Graphs:
We first choose node v uniformly at random. We then generate a random number x that is geometrically distributed
with mean pf /(1 − pf ). Node v selects x out-links incident
to nodes that were not yet visited. Let w1, w2, . . . , wx denote the other ends of these selected links. We then apply
this step recursively to each of w1, w2, . . . , wx until enough
nodes have been burned. As the process continues, nodes
cannot be visited a second time, preventing the construction
from cycling. If the fire dies, then we restart it, i.e. select
new node v uniformly at random. We call the parameter pf
the forward burning probability.
'''

#1. choose first node v uniformly at random and store it
v_new = np.random.randint(len(adj))
nodes = torch.tensor([v_new])
print('nodes: ', nodes)
#2. generate random number x from geometrix distribution with mean pf/(1-pf)
pf=0.3 #burning probability, evaluated as best from the given paper
x = np.random.geometric(pf/(1-pf))

#3. let idx choose x out-links
w = (adj[v_new]==1).nonzero()
if w.shape[0]>x:
    idx_w = random.sample(range(0, w.shape[0]), x)    
    w=w[idx_w]
    
#4. loop until 15% of the dataset is covered

while len(nodes)<20:
    v_new = w[0].item()
    w = (adj[v_new]==1).nonzero()
    for i in w:
        for j in nodes:
            if w[i]==nodes[j]:
                w[i]=0
    w = w.remove(0)
    if w.shape[0]>x:
        idx_w = random.sample(range(0, w.shape[0]), x)    
    v_new = torch.tensor([v_new])
    nodes = torch.cat((nodes,v_new),0)
    print(nodes)

nodes:  tensor([0])


IndexError: index 0 is out of bounds for dimension 0 with size 0

In [35]:
num_nodes = A.shape[0]
num_edges = A.sum()

# Convert adjacency matrix to a CUDA Tensor
adj = torch.FloatTensor(A.toarray()).cuda()

In [36]:
#torch.manual_seed(123)
# Define the embedding matrix
embedding_dim = 64
emb = nn.Parameter(torch.empty(num_nodes, embedding_dim).normal_(0.0, 1.0))

# Initialize the bias
# The bias is initialized in such a way that if the dot product between two embedding vectors is 0 
# (i.e. z_i^T z_j = 0), then their connection probability is sigmoid(b) equals to the 
# background edge probability in the graph. This significantly speeds up training
edge_proba = num_edges / (num_nodes**2 - num_nodes)
bias_init = np.log(edge_proba / (1 - edge_proba))
b = nn.Parameter(torch.Tensor([bias_init]))


# Regularize the embeddings but don't regularize the bias
# The value of weight_decay has a significant effect on the performance of the model (don't set too high!)
opt = torch.optim.Adam([
    {'params': [emb], 'weight_decay': 1e-7}, {'params': [b]}],
    lr=1e-2)


In [37]:
def compute_loss_ber_sig(adj, emb, b=0.1): 
    #kernel: theta(z_i,z_j)=sigma(z_i^Tz_j+b)
    # Initialization
    N,d=emb.shape
    
    #compute f(z_i, z_j) = sigma(z_i^Tz_j+b)
    dot=torch.matmul(emb,emb.T)
    logits =dot+b
    
    #transform adj
    ind=torch.triu_indices(N,N,offset=1)
    logits = logits[ind[0], ind[1]] 
    labels = adj[ind[0],ind[1]]
    
    
    #compute p(A|Z)
    loss = F.binary_cross_entropy_with_logits(logits, labels, weight=None, size_average=None, reduce=None, reduction='mean')
    return loss

def compute_loss_d1(adj, emb, b=0.0): 
    """Compute the rdf distance of the Bernoulli model."""
    # Initialization
    start_time = time.time()
    N,d=emb.shape
    squared_euclidian = torch.zeros(N,N).cuda()
    gamma= 0.1
    end_time= time.time()
    duration= end_time -start_time
    #print(f' Time for initialization = {duration:.5f}')
    # Compute squared euclidian
    start_time = time.time()
    for index, embedding in enumerate(emb):
        sub =  embedding-emb + 10e-9
        squared_euclidian[index,:]= torch.sum(torch.pow(sub,2),1)
    end_time= time.time()
    duration= end_time -start_time
    #print(f' Time for euclidian = {duration:.5f}')
    # Compute exponentianl
    start_time = time.time()
    radial_exp = torch.exp (-gamma * torch.sqrt(squared_euclidian))
    loss = F.binary_cross_entropy(radial_exp, adj, reduction='none')
    loss[np.diag_indices(adj.shape[0])] = 0.0
    end_time= time.time()
    duration= end_time -start_time
    #print(f' Time for loss  = {duration:.5f}')
    return loss.mean()


def compute_loss_ber_exp2(adj, emb, b=0.1):
    #Init
    N,d=emb.shape

    #get indices of upper triangular matrix
    ind=torch.triu_indices(N,N,offset=1)
    
    #compute f(z_i, z_j) = sigma(z_i^Tz_j+b)
    dot=torch.matmul(emb,emb.T)
    print('dist: ', dot, dot.size(), type(dot))
    logits=1-torch.exp(-dot)
    logits=logits[ind[0],ind[1]]
    labels = adj[ind[0],ind[1]]
    print('logits: ', logits, logits.size(), type(logits))
    
    #compute loss
    loss = F.binary_cross_entropy_with_logits(logits, labels, reduction='mean')

    return loss

def compute_loss_KL(adj, emb, b=0.0):
    #adj = torch.FloatTensor(A.toarray()).cuda()
    degree= torch.from_numpy(adj.sum(axis=1))
    print('degree: ', degree, type(degree), degree.size())
    inv_degree=torch.diagflat(1/degree).cuda()
    print('invdegree: ', invdegree, type(invdegree), invdegree.size())
    P = inv_degree.mm(adj) 
    print('P: ', invdegree, type(invdegree), invdegree.size())
    loss = -(P*torch.log( 10e-9+ F.softmax(emb.mm(emb.t() ),dim=1,dtype=torch.float)))
    return loss.mean()

In [38]:
max_epochs = 1000
display_step = 250
compute_loss = compute_loss_KL

for epoch in range(max_epochs):
    opt.zero_grad()
    loss = compute_loss(adj, emb, b)
    loss.backward()
    opt.step()
    # Training loss is printed every display_step epochs
    if epoch == 0 or (epoch + 1) % display_step == 0:
        print(f'Epoch {epoch+1:4d}, loss = {loss.item():.5f}')

TypeError: expected np.ndarray (got Tensor)